# Reformat catalog for CDS upload
Setup catalog info below

In [1]:
region_name = "ALL_10-800" #"CAINGSGII_10-800" #"TDS" #"CAINGSGII_10-800" #"WD" #"MDIS_10-800" #"TDS" #  _ELAISN1
region_dir_name = "./vasca_pipeline/"+region_name+"/"
region_cat_fname = region_dir_name+"region_"+region_name+"_cat.fits"
print_latex = True
tabnames_4CDS = ["tt_sources","tt_detections","tt_fields","tt_visits", "tt_coadd_sources"]

In [2]:
%matplotlib widget
import os
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
from scipy.stats import chi2

from regions import CircleSkyRegion
from astropy import units as uu
from astropy.coordinates import SkyCoord
from astropy.wcs.utils import pixel_to_skycoord
from astropy.table import Table,Column
import numpy as np
from matplotlib.ticker import ScalarFormatter

from vasca.region import Region
from vasca.field import BaseField
import vasca.visualization as vvis

from vasca.utils import mag2flux, flux2mag, select_obs_filter,freq2period,period2freq,flux2mag_np,mag2flux_np, dd_filter2id,get_flat_table
from vasca.tables_dict import dd_vasca_columns
import cdspyreadme

rc = Region()
rc.load_from_fits(region_cat_fname)

In [3]:
#Add Object group to source table
rc.copy_table_columns("tt_sources","tt_simbad",["otype","ogrp", "main_id"],"simbad_match_id")
#rc.tt_sources.rename_column("match_distance","match_distance_simbad"),"match_distance"
rc.tt_sources.rename_column("ogrp","ogrp_simbad")
rc.tt_sources.rename_column("otype","otype_simbad")
rc.tt_sources.rename_column("main_id","main_id_simbad")

copy_cols = ["Source"] #,,"Plx_dist","AG","RPlx", ,"AG",,"RPlx",,"Plx"   ,"BP-RP","Gmag_abs"
rc.copy_table_columns("tt_sources","tt_gaiadr3",copy_cols,"gaiadr3_match_id")
for col in copy_cols:
    rc.tt_sources.rename_column(col,col+"_gaiadr3")

rc.copy_table_columns("tt_sources","tt_gaiaedr3_wd",["WDJname"],"gaiaedr3_wd_match_id") # ,"Pwd"
rc.tt_sources.rename_column("WDJname","WDJname_gaiaedr3_wd")
#rc.tt_sources.rename_column("Pwd","Pwd_gaiaedr3_wd")

rc.copy_table_columns("tt_sources","tt_gfcat",["gfcat_objid"],"gfcat_src_id")
rc.tt_sources.rename_column("gfcat_objid","objid_gfcat")

rc.tt_sources.remove_columns(["gaiaedr3_wd_match_id","simbad_match_id","gaiadr3_match_id","gfcat_src_id"]) 

rc.copy_table_columns("tt_sources","tt_lombscargle",["ls_peak_pval", "ls_model_rchiq"],"rg_src_id")

### Print out LATEX for paper tables

In [5]:
#Latex table names print out and tt_sources  columns
if print_latex:
    for tabname in tabnames_4CDS:
        new_tab_name = tabname.replace("tt_","").upper()
        print(new_tab_name.replace("_","\_"), "&  \\\\")
    
    cat_cols = ['src_name','nr_det', 'ra', 'dec', 'pos_err', 'pos_xv', 'pos_var', 'pos_cpval', 'pos_rchiq', 'obs_filter_id',
                'sel', 'flux', 'flux_err', 'flux_nxv', 'flux_var', 'flux_cpval', 'flux_rchiq', 'coadd_ffactor', 'coadd_fdiff_s2n',
                'rg_src_id', 'nr_fd_srcs', 'hr', 'hr_err',"ogrp_simbad","otype_simbad","main_id_simbad","Source_gaiadr3","WDJname_gaiaedr3_wd","objid_gfcat","ls_peak_pval", "ls_model_rchiq"]
    print("\n")
    for col in cat_cols:
        icol = dd_vasca_columns[col.replace("_simbad","").replace("_gaiadr3","").replace("_gaiaedr3_wd","").replace("objid_gfcat","gfcat_objid")]
        print(icol["name"].replace("_","\_").upper(),"&",icol["description"],"&",icol["unit"].replace("1",""), "\\\\")

SOURCES &  \\
DETECTIONS &  \\
FIELDS &  \\
VISITS &  \\
COADD\_SOURCES &  \\


SRC\_NAME & VASCA catalog source name &  \\
NR\_DET & Number of detections &  \\
RA & Sky coordinate Right Ascension (J2000) & degree \\
DEC & Sky coordinate Declination (J2000) & degree \\
POS\_ERR & Sky coordinate position error & arcsec \\
POS\_XV & Sky position excess variance & arcsec2 \\
POS\_VAR & Sky position variance & arcsec2 \\
POS\_CPVAL & Sky position quality &  \\
POS\_RCHIQ & Sky position reduced chisquared of the constant mean &  \\
OBS\_FILTER\_ID & Observation filter ID number &  \\
SEL & Selection of rows for VASCA analysis &  \\
FLUX & Flux density & e-6Jy \\
FLUX\_ERR & Flux density error & e-6Jy \\
FLUX\_NXV & Flux normalized excess variance &  \\
FLUX\_VAR & Flux variance & e-2Jy2 \\
FLUX\_CPVAL & Probability value for a constant flux from the chisquare test &  \\
FLUX\_RCHIQ & Flux reduced chisquared of the constant mean &  \\
COADD\_FFACTOR & Source flux divided by flux of the assoc

KeyError: 'wdjname'

### Rename table columns, remove unwanted columns and store into CDS table

In [ ]:
#print(rc.tt_sources.colnames)
tablemaker = cdspyreadme.CDSTablesMaker()


for tabname in tabnames_4CDS:
    print(tabname)
    tt = get_flat_table(rc.__dict__[tabname])
    for colname in tt.colnames:
        new_col_name = colname.upper()
        if new_col_name not in tt.colnames:
            tt.rename_column(colname,new_col_name)


        #tt.rename_column("MAINCAT_MATCH_ID","WDMAIN_MATCH_ID")

# rc.tt_sources.remove_columns(["GFCAT_DIST","FD_SRC_ID","COADD_DIST","SEL","RG_FD_ID"])
    if "RA" in tt.colnames:
        if tabname == "tt_gaiadr3":
            tt.rename_column("RAJ2000","RA_GAIA")
            tt.rename_column("DEJ2000","DEC_GAIA")
            
        tt.rename_column("RA","RAJ2000")
        tt.rename_column("DEC","DECJ2000")
    remove_cols = ["GFCAT_DIST","COADD_DIST","SEL","RG_FD_ID"]
    if tabname not in ["tt_src_id_map", "tt_detections", "tt_coadd_detections"]:
        remove_cols.append("FD_SRC_ID")
    for rcol in remove_cols:
        if rcol in tt.colnames:
            tt.remove_column(rcol) 
    display(tt)

    tablemaker.addTable(tt, name=tabname.replace("tt_","").upper())

In [ ]:
os.chdir(region_dir_name+"/CDS")
tablemaker.writeCDSTables()
with open("ReadMe", "w") as fd:
    tablemaker.makeReadMe(out=fd)

In [ ]:
display(rc.tt_simbad[0:2])
display(rc.tt_gaiadr3[0:2])
display(rc.tt_gfcat[0:2])
display(rc.tt_gaiaedr3_wd[0:2])
display(rc.tt_sources[0:2])